In [6]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [7]:
AGENT_NAME = "ab020-streaming"

In [8]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='fab911e3-47c5-43f2-9942-382a98d45975', created_at=datetime.datetime(2025, 7, 27, 0, 52, 55, 493055, tzinfo=TzInfo(UTC)), name='69297be2-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 0, 52, 55, 493055, tzinfo=TzInfo(UTC)))


In [ ]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='6f1c066c-f400-49a5-a8bd-afe0f2367b4c', agent_id='ef21f2e7-5bbe-440a-824b-7e89f462a781', sequence_id=203, task_id='fab911e3-47c5-43f2-9942-382a98d45975', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 0, 52, 56, 693376, tzinfo=TzInfo(UTC)))


In [10]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/27/2025 00:52:56] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 00:52:56] ─────────────────────────╮
│ Hello! I can assist you with a variety of tasks, such as:                    │
│                                                                              │
│  1 Answering questions on a wide range of topics, including science,         │
│    history, technology, and more.                                            │
│  2 Providing explanations and summaries of complex concepts.                 │
│  3 Assisting with language translations and grammar checks.                  │
│  4 Offering writing tips, editing assistance, and brainstorming ideas for    │
│    creative projects.                                                        │
│  5 Giving advice on studying, time management, and organization.             │
│  6 Sharing information about books, movies, and other media.                 │
│  7 Engaging in general conversation or providing entertainment.              │
│                                                                              │
│ If you have a specific question or need help with something, feel free to    │
│ ask!                                                                         │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
